In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('Cleaned-P&P data.csv')

blanks = []  # start with an empty list`

for i,c,d,inc,sd in df1.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
df1.drop(blanks, inplace=True)
df1["desc"] = df1["short_des"] + '. ' + df1["desc"]
df1.reset_index(drop=True,inplace=True)


In [3]:

df2 = pd.read_csv('Cleaned-P&P data.csv')
blanks2 = []  # start with an empty list`

for i,c,d,inc,sd in df2.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks2.append(i)     # add matching index numbers to the list
df2.drop(blanks2, inplace=True)
df2.reset_index(drop=True,inplace=True)

In [7]:
dataset = pd.concat([df1, df2], ignore_index=True)

In [195]:
# Cleaning the texts
import re
corpus = []
all_words = []
max_len=0
for i in range(0, 488):

    review = re.sub('\w\d{8}', 'Adam', dataset['desc'][i])
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    
    review = nlp(review)
    review = [word.text for word in review]
   # for word.text in review:
        
    all_words.append(review) 
    if len(review) > max_len:
        max_len = len(review)
    ds = ' '.join(review)
#    for word2vec we want an array of vectors
    corpus.append(ds)
    

### Standardise key terms in data

In [64]:
dpf = pd.read_csv('Del-Pass.csv')



In [67]:
dpf['kpattern']

(13,)

In [151]:
keys_list = dpf['kpattern'].tolist()
regex = re.compile(r'(' + '|'.join(keys_list) + r')') 

In [117]:
cus_list = ['account holder','customers','customer', 'cust']
regey = re.compile(r'(' + '|'.join(cus_list) + r')') 

In [196]:
cldict = {'P':1 , 'R':0}
dataset['Class Label'] = dataset['class'].map(cldict)
label_series = pd.Series(dataset['Class Label'])

In [197]:
from keras.utils import to_categorical

class_label = np.array(label_series)
y = class_label
y.shape

(488,)

In [198]:
X = corpus

In [199]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=47)


### Create custom rules 

In [209]:
class SegmentFeaturizer:
    def __init__(self):
        pass
    
         
    def featurize(self, seg):
#         refined_corpus=[]
        
#         refined_corpus.append(rf_cps)
#     def seg(self,seg):
                       
        return regex.sub(lambda m: m.group().replace(m.group(),"del-pass"), seg)

In [210]:
segment_featurizer = SegmentFeaturizer()  # more on this below
class CustomLinguisticFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass   
    def fit(self, x, y=None):
        return self    
    def transform(self, data):
        return segment_featurizer.featurize(data)

### Spacy Vector Transformer

In [202]:

from sklearn.base import BaseEstimator, TransformerMixin
class SpacyVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300

    def fit(self, X, y):
        return self

    def transform(self, X):
        # Doc.vector defaults to an average of the token vectors.
        # https://spacy.io/api/doc#vector
        return [self.nlp(text).vector for text in X]

### Reduce dimensionality

In [191]:
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

embeddings_pipeline = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", RandomForestClassifier()),
    ]
)
embeddings_pipeline.fit(X_train, y_train)

C:\Users\AVNEET\miniconda3\envs\nlp_course\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('mean_embeddings', SpacyVectorTransformer(nlp=<spacy.lang.en.English object at 0x000001E036772308>)), ('reduce_dim', TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, c...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

embeddings_pipeline2 = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", LinearSVC()),
    ]
)
embeddings_pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('mean_embeddings', SpacyVectorTransformer(nlp=<spacy.lang.en.English object at 0x000001E036772308>)), ('reduce_dim', TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)), ('classifier', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [20]:
from sklearn.calibration import CalibratedClassifierCV
svm = LinearSVC()
embeddings_pipeline3 = Pipeline(
    steps=[
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", CalibratedClassifierCV(svm)),
    ]
)
embeddings_pipeline3.fit(X_train, y_train)

C:\Users\AVNEET\miniconda3\envs\nlp_course\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Pipeline(memory=None,
     steps=[('mean_embeddings', SpacyVectorTransformer(nlp=<spacy.lang.en.English object at 0x000001E036772308>)), ('reduce_dim', TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)), ('classifier', CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, clas...lty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv='warn', method='sigmoid'))])

In [211]:
from sklearn.calibration import CalibratedClassifierCV
svm = LinearSVC()
embeddings_pipeline4 = Pipeline(
    steps=[
        ("cust",CustomLinguisticFeatureTransformer()),
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", CalibratedClassifierCV(svm)),
    ]
)
embeddings_pipeline4.fit(X_train, y_train)

TypeError: expected string or bytes-like object

In [21]:

y_pred = embeddings_pipeline3.predict(X)
cr = classification_report(y, y_pred)

In [22]:
print(cr)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       190
           1       1.00      0.99      1.00       298

   micro avg       1.00      1.00      1.00       488
   macro avg       0.99      1.00      1.00       488
weighted avg       1.00      1.00      1.00       488



In [79]:
list1 = ['remove delivery pass on customer a012345678']
print(embeddings_pipeline3.predict(list1))
probs = embeddings_pipeline3.predict_proba(list1)
np.around(probs, decimals = 3)*100

[1]


array([[23.4, 76.6]])

In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y,y_pred)

array([[190,   0],
       [  2, 296]], dtype=int64)

In [25]:
accuracy_score(y, y_pred)

0.9959016393442623

In [184]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

0.0

In [88]:
from sklearn.model_selection import cross_val_score
cross_val_score(embeddings_pipeline3, X, y, cv = 10)

KeyboardInterrupt: 

In [ ]:
cv_mean_score = np.mean(cross_val_score(embeddings_pipeline3, X, y, cv = 10))
clf_score = embeddings_pipeline3.score(X_test, y_test)
cv_mean_score, clf_score

In [93]:
predictions = embeddings_pipeline3.predict(X)
len(X)

488

In [180]:
clf_score = embeddings_pipeline3.score(X, y)
clf_score

0.9959016393442623

In [181]:
Faulty_Xtest =[]
for i in range(0,488):
    if y[i] != predictions[i]:
        print(f'predicted class:{X[i]}', f'Index is : {i}')
        Faulty_Xtest.append(X[i])
arr = np.array(Faulty_Xtest)
arr

predicted class:delivery pass needs removing on staff accounts supra dba req please can the delivery pass be removed from the attached staff accounts thank you Index is : 201
predicted class:please can the delivery pass be removed from the attached staff accounts thank you Index is : 445


array(['delivery pass needs removing on staff accounts supra dba req please can the delivery pass be removed from the attached staff accounts thank you',
       'please can the delivery pass be removed from the attached staff accounts thank you'],
      dtype='<U143')

In [102]:
dataset['inc_num'][201], dataset['desc'][201], dataset['class'][201]

('INC0612636',
 'delivery pass needs removing on staff accounts supra dba req. please can the delivery pass be removed from the attached staff accounts?thank you!',
 'P')

In [11]:
# Enter your code here:

for token in nlp(X_test[94]):
    print(f'{token.text:{12}} {token.pos_:{6}} {token.dep_:{6}} {spacy.explain(token.dep_)}')

hi           INTJ   intj   interjection
the          DET    det    determiner
customer     NOUN   nsubj  nominal subject
took         VERB   ccomp  clausal complement
the          DET    det    determiner
delivery     NOUN   compound None
subscription NOUN   dobj   direct object
out          PART   prt    particle
on           ADP    prep   prepositional modifier
th           DET    nummod None
november     NOUN   pobj   object of preposition
but          CCONJ  cc     coordinating conjunction
is           VERB   aux    auxiliary
still        ADV    advmod adverbial modifier
being        VERB   auxpass auxiliary (passive)
charged      VERB   conj   conjunct
delivey      NOUN   advmod adverbial modifier
please       INTJ   intj   interjection
advise       VERB   ROOT   None
thanks       NOUN   dobj   direct object
peter        X      dobj   direct object


In [216]:

from spacy import displacy
displacy.render(nlp(X_test[95]), style='ent', jupyter=True)

In [26]:
import re
for match in re.finditer("delivery subscription ",X_test[94]):
    X_test[94]
    print(match.span())


(25, 47)


In [24]:
match.span()[1]

47

In [31]:
matche = re.findall("delivery subscription ",X_test[94])
match.span()

(25, 47)

In [27]:
regex = re.compile('([a-zA-Z]\"[a-zA-Z])', re.S)
myfile =  'foo"s bar'
myfile2 = regex.sub(lambda m: m.group().replace('"',"%",1), myfile)
print(myfile2)

foo%s bar


In [146]:


lst = ['foo', 'bar', 'and', 'delivery subscription']

def replace_double_quote(match):
    text = match.group()
    print(match.group())
    return text.replace(text, 'rap')
regex = re.compile(r'(' + '|'.join(lst) + r')')
myfile = 'foo"s bar and bar"s foo delivery subscription why'
regex.sub(replace_double_quote, myfile)



foo
bar
and
bar
foo
delivery subscription


'rap"s rap rap rap"s rap rap why'

In [140]:


lst = ['foo', 'bar', 'and']
def my_func():
    
    regex = re.compile(r'(' + '|'.join(lst) + r')')
    myfile = 'foo"s bar and bar"s foo'
   
    pxk = regex.sub(lambda m: m.group().replace(m.group(),"rap",1), myfile)
    return pxk
#     def replace_double_quote(match):
#         text = match.group()
#         print(text)
#         return text.replace(text, 'rap')
   
print(my_func())

frap"s brap arap brap"s frap


98

In [168]:
seg = "customer need to have subscription removed from his account A01234567"

def match_kpatterns(seg):
    
    return re.sub('\w\d{8}', 'Adam', seg)
def r_featurize(seg):
#         refined_corpus=[]
    rfseg = match_kpatterns(seg)

#         refined_corpus.append(rf_cps)
#     def seg(self,seg):

    return regex.sub(lambda m: m.group().replace(m.group(),"del-pass"), rfseg)
r_featurize(seg) 

'customer need to have del-pass removed from his account Adam'

In [179]:
X_train[0]

'please can you apply the annual delivery subscription is activated on this customers account b purchased still being charged for delivery thank you'